ASAP CRN Metadata validation - wave 2

# Team Jakobsson. ASAP CRN Metadata validation - wave 2

28 Nov 2023
Andy Henrie




In [1]:
import pandas as pd

from pathlib import Path

from utils.io import read_meta_table, get_dtypes_dict, ReportCollector
from utils.qcutils import prep_table, validate_table, validate_file, read_ASAP_CDE, reorder_table_to_CDE



%load_ext autoreload
%autoreload 2

Streamlit NOT successfully


## copy metadata locally


In [3]:
!gsutil -u dnastack-asap-parkinsons cp -r "gs://asap-raw-data-team-jakobsson/CRN_snRNA/metadata/*.csv" "/Users/ergonyc/Projects/ASAP/team-jakobsson/metadata/"

AccessDeniedException: 403 admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com does not have storage.objects.list access to the Google Cloud Storage bucket. Permission 'storage.objects.list' denied on resource (or it may not exist).


In [4]:
metadata_version = "v2"

CDE_df,dtypes_dict = read_ASAP_CDE(metadata_version=metadata_version)
METADATA_VERSION_DATE = f"{metadata_version}_{pd.Timestamp.now().strftime('%Y%m%d')}"


## convert 
data_path = Path.home() / ("Projects/ASAP/team-jakobsson")
metadata_path = data_path / "metadata"


SUBJECT = read_meta_table(metadata_path / "SUBJECT.csv", dtypes_dict)
CLINPATH = read_meta_table(metadata_path / "CLINPATH.csv", dtypes_dict)
STUDY = read_meta_table(metadata_path / "STUDY.csv", dtypes_dict)
PROTOCOL = read_meta_table(metadata_path / "PROTOCOL.csv", dtypes_dict)
SAMPLE = read_meta_table(metadata_path / "SAMPLE.csv", dtypes_dict)
DATA = read_meta_table(metadata_path / "DATA.csv", dtypes_dict)

# SUBJECT = pd.read_csv(f"{metadata_path}/SUBJECT.csv")
# CLINPATH = pd.read_csv(f"{metadata_path}/CLINPATH.csv")
# STUDY = pd.read_csv(f"{metadata_path}/STUDY.csv")
# PROTOCOL = pd.read_csv(f"{metadata_path}/PROTOCOL.csv")
# SAMPLE = pd.read_csv(f"{metadata_path}/SAMPLE.csv")
# DATA = pd.read_csv(f"{metadata_path}/DATA.csv")


read url


In [5]:

export_root = Path.cwd() / "clean/team-Jakobsson"
if not export_root.exists():
    export_root.mkdir(parents=True, exist_ok=True)

STUDY.to_csv( export_root / "STUDY.csv")
PROTOCOL.to_csv(export_root / "PROTOCOL.csv")
SAMPLE.to_csv(export_root / "SAMPLE.csv")
SUBJECT.to_csv(export_root / "SUBJECT.csv")
CLINPATH.to_csv(export_root / "CLINPATH.csv")
DATA.to_csv(export_root / "DATA.csv")


In [6]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,ASAP1_PD_NP16-162_SN,1,1,0,1,fastq,ASAP1_PD_NP16-162_SN_R1.fastq.gz,Barcode & UMI sequence,cb8b16ec040ba7b7f18b446eb8135500,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
1,ASAP2_PD_NP16-25_SN,1,1,0,1,fastq,ASAP2_PD_NP16-25_SN_R1.fastq.gz,Barcode & UMI sequence,380812c09848e08dbd5035f8cf3686a8,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
2,ASAP3_PD_P73_SN,1,1,0,1,fastq,ASAP3_PD_P73_SN_R1.fastq.gz,Barcode & UMI sequence,604748473940c0e3a7441314d85b6550,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
3,ASAP4_PD_P74_SN,1,1,0,1,fastq,ASAP4_PD_P74_SN_R1.fastq.gz,Barcode & UMI sequence,86c1d5232907d35b4e629d7c769514ef,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
4,ASAP5_Ctl_NP16-161_SN,1,1,0,1,fastq,ASAP5_Ctl_NP16-161_SN_R1.fastq.gz,Barcode & UMI sequence,0d417eb4629bbddd8299de991f0e1883,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>


In [7]:
!pwd

/Users/ergonyc/Projects/ASAP/meta-clean


In [8]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/jakobsson-credentials.json  
!gsutil -u dnastack-asap-parkinsons hash -h "gs://asap-raw-data-team-jakobsson/CRN_snRNA/**/*.gz" > jakobsson_hexhash.log

Activated service account credentials for: [raw-admin-jakobsson@dnastack-asap-parkinsons.iam.gserviceaccount.com]

using the module's C extension, so checksumming will run very slowly. For help
installing the extension, please see "gsutil help crcmod".



In [19]:
# !gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/jakobsson-credentials.json  
# !gsutil -u dnastack-asap-parkinsons ls "gs://asap-raw-data-team-jakobsson/CRN_snRNA/**/*" > manifest.txt

Activated service account credentials for: [raw-admin-jakobsson@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [9]:

from utils.checksums import extract_md5_from_details, extract_md5_from_details2


In [10]:
bucket_files_md5 = extract_md5_from_details2("jakobsson_hexhash.log")
bucket_files_md5

{'ASAP10_PFC_count_matrix.tar.gz': '9d9825c4260f2ce9e2886151865640df',
 'ASAP11_PFC_count_matrix.tar.gz': '898b81becfa49b1a9a425c7748422ab6',
 'ASAP12_PFC_count_matrix.tar.gz': '6384e6b377571fb5d5b9407fc210bcfa',
 'ASAP13_PFC_count_matrix.tar.gz': '31ac48621143a6a347ffc960c305aa81',
 'ASAP14_PFC_count_matrix.tar.gz': 'f7a371fa2cbf4c89344cfd72638d4f67',
 'ASAP15_PFC_count_matrix.tar.gz': '1d66933bd2192ee477c5000fbbb72160',
 'ASAP16_PFC_count_matrix.tar.gz': '773cb9c9a42fc758f7f61208b0455f5f',
 'ASAP17_PFC_count_matrix.tar.gz': '8d1ca3cedf51b283c94550b6b7bdfde8',
 'ASAP18_AM_count_matrix.tar.gz': '5fa973b880bfa81b98c7ec0a12974058',
 'ASAP19_PFC_count_matrix.tar.gz': '683b5ead3e4105b7a8694664fad9899f',
 'ASAP1_SN_count_matrix.tar.gz': '7978bc4c35f3166a5ccbce94b91ca2c9',
 'ASAP20_AM_count_matrix.tar.gz': '95384e11a445690a2955ae19c20f6fdc',
 'ASAP21_SN_count_matrix.tar.gz': '53a91cba0bd2077c236bd1f05b88c959',
 'ASAP22_PFC_count_matrix.tar.gz': '728e87e88cc4af99595a0f21c55f7e1f',
 'ASAP24_AM

In [11]:



checksum = DATA[['file_name','file_MD5']]
checksum['check1'] = checksum['file_MD5'].str.strip()
checksum['check2'] = checksum['file_name'].map(bucket_files_md5)



/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_53755/2412817574.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check1'] = checksum['file_MD5'].str.strip()
/var/folders/0z/hvx6j8cn6yj3bqbyq6rtsxm40000gn/T/ipykernel_53755/2412817574.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  checksum['check2'] = checksum['file_name'].map(bucket_files_md5)


In [12]:
checksum[checksum.check1 != checksum.check2].file_name.to_list()


[]

In [44]:
DATA.head()

,sample_id,replicate,replicate_count,repeated_sample,batch,file_type,file_name,file_description,file_MD5,technology,omic,adjustment,content,time,header,annotation,configuration_file
0,ASAP1_PD_NP16-162_SN,1,1,0,1,fastq,ASAP1_PD_NP16-162_SN_R1.fastq.gz,Barcode & UMI sequence,cb8b16ec040ba7b7f18b446eb8135500,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
1,ASAP2_PD_NP16-25_SN,1,1,0,1,fastq,ASAP2_PD_NP16-25_SN_R1.fastq.gz,Barcode & UMI sequence,380812c09848e08dbd5035f8cf3686a8,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
2,ASAP3_PD_P73_SN,1,1,0,1,fastq,ASAP3_PD_P73_SN_R1.fastq.gz,Barcode & UMI sequence,604748473940c0e3a7441314d85b6550,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
3,ASAP4_PD_P74_SN,1,1,0,1,fastq,ASAP4_PD_P74_SN_R1.fastq.gz,Barcode & UMI sequence,86c1d5232907d35b4e629d7c769514ef,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>
4,ASAP5_Ctl_NP16-161_SN,1,1,0,1,fastq,ASAP5_Ctl_NP16-161_SN_R1.fastq.gz,Barcode & UMI sequence,0d417eb4629bbddd8299de991f0e1883,SN,RNA,Raw,Reads,0,<NA>,<NA>,<NA>


## transfer metadata Jakobsson 

In [13]:
# Jakobsson
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/jakobsson-credentials.json 


Activated service account credentials for: [raw-admin-jakobsson@dnastack-asap-parkinsons.iam.gserviceaccount.com]


## fix bucket file structure to match the prescribed convention

remove "/CRN_snRNA/" from path

In [ ]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-jakobsson/metadata/v2"

# clean up errant metadata.


      2891  2023-11-28T23:57:58Z  gs://asap-raw-data-team-jakobsson/metadata/v2/CLINPATH.csv#1701215878806977  metageneration=1
     64824  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/DATA.csv#1701215879761241  metageneration=1
      1742  2023-11-28T23:58:00Z  gs://asap-raw-data-team-jakobsson/metadata/v2/PROTOCOL.csv#1701215880248740  metageneration=1
     17848  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/SAMPLE.csv#1701215879496668  metageneration=1
      2494  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/STUDY.csv#1701215879970332  metageneration=1
      1958  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/SUBJECT.csv#1701215879255400  metageneration=1
TOTAL: 6 objects, 91757 bytes (89.61 KiB)


In [14]:

!gsutil -u dnastack-asap-parkinsons ls -al "gs://asap-raw-data-team-jakobsson/metadata/v2"

# clean up errant metadata.


      2891  2023-11-28T23:57:58Z  gs://asap-raw-data-team-jakobsson/metadata/v2/CLINPATH.csv#1701215878806977  metageneration=1
     64824  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/DATA.csv#1701215879761241  metageneration=1
      1742  2023-11-28T23:58:00Z  gs://asap-raw-data-team-jakobsson/metadata/v2/PROTOCOL.csv#1701215880248740  metageneration=1
     17848  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/SAMPLE.csv#1701215879496668  metageneration=1
      2494  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/STUDY.csv#1701215879970332  metageneration=1
      1958  2023-11-28T23:57:59Z  gs://asap-raw-data-team-jakobsson/metadata/v2/SUBJECT.csv#1701215879255400  metageneration=1
TOTAL: 6 objects, 91757 bytes (89.61 KiB)


In [15]:

!gsutil -u dnastack-asap-parkinsons cp "./clean/team-Jakobsson/*.csv"  "gs://asap-raw-data-team-jakobsson/metadata/v2/"

Copying file://./clean/team-Jakobsson/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/SUBJECT.csv [Content-Type=text/csv]...    
Copying file://./clean/team-Jakobsson/SAMPLE.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Jakobsson/DATA.csv [Content-Type=text/csv]...       
- [4 files][ 87.1 KiB/ 87.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Jakobsson/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/PROTOCOL.csv [Content-Type=text/csv]...   
\ [6 files][ 91.2 KiB/ 91.2 KiB]                                                
Operation completed over 6 objects/91.2 KiB.                                     


### copy to workflow-dev bucket

First copy each set of metadata locally ...

In [16]:
!gcloud auth activate-service-account --key-file=/Users/ergonyc/Projects/ASAP/wf-credentials.json

Activated service account credentials for: [admin-workflow-dev@dnastack-asap-parkinsons.iam.gserviceaccount.com]


In [17]:
!gsutil cp "./clean/team-Jakobsson/*.csv" "gs://asap-workflow-dev/metadata/v2/jakobsson"

Copying file://./clean/team-Jakobsson/CLINPATH.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/SUBJECT.csv [Content-Type=text/csv]...    
Copying file://./clean/team-Jakobsson/SAMPLE.csv [Content-Type=text/csv]...     
Copying file://./clean/team-Jakobsson/DATA.csv [Content-Type=text/csv]...       
- [4 files][ 87.1 KiB/ 87.1 KiB]                                                
==> NOTE: You are performing a sequence of gsutil operations that may
run significantly faster if you instead use gsutil -m cp ... Please
see the -m section under "gsutil help options" for further information
about when gsutil -m can be advantageous.

Copying file://./clean/team-Jakobsson/STUDY.csv [Content-Type=text/csv]...
Copying file://./clean/team-Jakobsson/PROTOCOL.csv [Content-Type=text/csv]...   
\ [6 files][ 91.2 KiB/ 91.2 KiB]                                                
Operation completed over 6 objects/91.2 KiB.                                     


In [18]:
!gsutil ls -al "gs://asap-workflow-dev/metadata/v2"

                                 gs://asap-workflow-dev/metadata/v2/hafler/
                                 gs://asap-workflow-dev/metadata/v2/hardy/
                                 gs://asap-workflow-dev/metadata/v2/jakobsson/
                                 gs://asap-workflow-dev/metadata/v2/lee/
                                 gs://asap-workflow-dev/metadata/v2/scherzer/
